<H1> Разработка инструмента для прогнозирования популярности постов в социальных сетях с применением методов машинного обучения </H1>

Проект содержит следующие этапы:
* Сбор БД и визуализация данных
* Токенизация, лемматизация и стемминг данных
* Векторизация
* **Классификация**
* Подбор гиперпараметров и демонстрация итоговых результатов

Для корректной работы кода необходимо загрузить файлы "ITMO_2.csv",  "ITMO_2_vectorized_bag_of_words.npz", полученные в рамках предыдущего шага проекта.

# **Разделение на обучающую и тестовую выборки**

In [ ]:
import numpy as np

# Загрузка данных из файла .npz
data = np.load('ITMO_2_vectorized_bag_of_words.npz', allow_pickle=True)

# Вывод списка переменных в файле
print("Переменные в файле:")
for variable_name in data.files:
    print(variable_name)

Переменные в файле:
X
feature_names


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Загрузка данных из файла .npz
data_npz = np.load('ITMO_2_vectorized_bag_of_words.npz', allow_pickle=True)

# Извлечение переменной 'data' из файла .npz
X_data = data_npz['X']

# Загрузка целевой переменной из файла ITMO_2.csv
y_data = pd.read_csv('ITMO_2.csv')['popular']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [ ]:
# размер обучающей и тестовой выборки
print(len(X_train))
print(len(X_test))

12501
3126


In [ ]:
# метрики качества
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# **Наивный Байес**
Наивный Байес рассматривает классификацию как вероятностную задачу - он вычисляет вероятность принадлежности объекта к каждому классу и выбирает тот класс, для которого вероятность максимальна.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf_nb = MultinomialNB(
    # alpha (default=1.0) - параметр регуляризации
    # Более высокие значения делают модель более устойчивой к переобучению
        alpha = 1.1
)

In [ ]:
clf_nb = clf_nb.fit(X_train, y_train)
y_predict_nb = clf_nb.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_nb))
print('Precision: ', precision_score(y_test, y_predict_nb, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_nb, average='macro'))
print('F1: ', f1_score(y_test, y_predict_nb, average='macro'))

Accuracy:  0.8218170185540628
Precision:  0.7001475531110789
Recall:  0.7069961308940109
F1:  0.7034393521925737


# **Случайный лес**
Случайный лес использует ансамблевое обучение, сочетающее в себе множество независимыо обученных решающих деревьев.



In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_rf =  RandomForestClassifier(
        n_estimators = 200, # количество деревьев в ансамбле (default=100)
        # какая функция используется для измерения качества разбиения деревьев
        criterion = 'gini', # gini - Индекс Джини, entropy - Энтропия, log_loss - логарифмическая потеря, default gini
        max_depth = 200, # максимальная глубина дерева, int, default=None
        # уменьшение дисбаланса классов
        # dict - пользовательские настройки
        # balanced - автоматическая балансировка классов
        # None - по умолчанию классы имеют одинаковые веса
        class_weight = 'balanced', # default=None
        random_state = 42 # контроль воспроизводимости результатов при использовании случайности
)

In [ ]:
clf_rf = clf_rf.fit(X_train, y_train)
y_predict_rf = clf_rf.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_rf))
print('Precision: ', precision_score(y_test, y_predict_rf, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_rf, average='macro'))
print('F1: ', f1_score(y_test, y_predict_rf, average='macro'))

Accuracy:  0.8442098528470889
Precision:  0.8172860873261675
Recall:  0.586899995772605
F1:  0.6058216806076463


# **Деревья решений**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf_dt =  DecisionTreeClassifier(
        # определяет метрику, по которой будет происходить разделение
        # 'gini' - индекс Джини (по умолчанию)
        # 'entropy' - энтропия
        # 'log_loss' - логарифмические потери
        criterion = 'entropy',
        # стратегия выбора признака
        # 'best' - лучшией признак значения (по умолчанию)
        # 'random' - выбирает случайный признак
        splitter = 'best',
        max_depth = 200, # максимальная глубина дерева, int, default=None
        # учет дисбаланса классов при обучении модели
        class_weight = 'balanced', #dict or ‘balanced’, default=None
        random_state = 42 # задает начальное значение генератора для воспроизводимости результатов
)

In [ ]:
clf_dt = clf_dt.fit(X_train, y_train)
y_predict_dt = clf_dt.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_dt))
print('Precision: ', precision_score(y_test, y_predict_dt, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_dt, average='macro'))
print('F1: ', f1_score(y_test, y_predict_dt, average='macro'))

Accuracy:  0.7869481765834933
Precision:  0.6487073298218808
Recall:  0.6614764419748449
F1:  0.6543837922367683


# **K-ближайших соседей**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf_KNN = KNeighborsClassifier(
        n_neighbors = 3, # int, default=5, кол-во ближайших соседей
        # способ взвешивания вклада соседей в определение класса
        # 'uniform' - все соседи имеют одинаковый вес
        # 'distance' - вес соседа прямо пропорционален его расстоянию до объекта
        weights = 'distance', # default=’uniform’
        # метрика расстояния, используемая для поиска
        # 'euclidean' - Евклидово расстояние
        # 'manhattan' - Манхэттенское расстояние
        # 'chebyshev' - расстояние Чебышева
        # 'cosine' - косинусное сходство
        # 'minkowski' - расстояние Минковского
        metric = 'euclidean', # default=’minkowski’
)

In [ ]:
clf_KNN = clf_KNN.fit(X_train, y_train)
y_predict_KNN = clf_KNN.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_KNN))
print('Precision: ', precision_score(y_test, y_predict_KNN, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_KNN, average='macro'))
print('F1: ', f1_score(y_test, y_predict_KNN, average='macro'))

Accuracy:  0.8119001919385797
Precision:  0.6377088914402347
Recall:  0.556108273982324
F1:  0.5616118175018698


# **Метод опорных векторов**

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
clf_lsvc = LinearSVC(
        # тип регуляризации
        # l1 - сумма абсолютных значений весов
        # l2 - сумма квадратов весов
        penalty = 'l2', # default=’l2’
        # коэффициент регуляризации; чем меньше, тем больше регуляризация
        C = 0.2, #float, default=1.0
        # способ учета дисбаланса классов
        class_weight = 'balanced', # default=None
        # начальное состояние генератора для воспроизводимости результата
        random_state = 42,
        # максимальное число итераций оптимизации
        max_iter = 2000 # int, default=1000
)

In [ ]:
clf_lsvc = clf_lsvc.fit(X_train, y_train)
y_predict_lsvc = clf_lsvc.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_lsvc))
print('Precision: ', precision_score(y_test, y_predict_lsvc, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_lsvc, average='macro'))
print('F1: ', f1_score(y_test, y_predict_lsvc, average='macro'))

Accuracy:  0.8234165067178503
Precision:  0.7021386534193432
Recall:  0.7065855191622272
F1:  0.7043069973713161


# **Логистическая регрессия**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_lr = LogisticRegression(
        # тип регуляризации
        # l1 - сумма абсолютных значений весов
        # l2 - сумма квадратов весов
        penalty = 'l2', # default=’l2’
        # коэффициент регуляризации; чем меньше, тем больше регуляризация
        C = 0.2, # float, default=1.0
        # способ учета дисбаланса классов
        class_weight = 'balanced', # default=None
        # начальное состояние генератора для воспроизводимости результата
        random_state = 42,
        # максимальное число итераций оптимизации
        max_iter = 100, # int, default=100
        # кол-во используемых ядер процессора
        n_jobs = -1, # int, default=None
)

In [ ]:
clf_lr = clf_lr.fit(X_train, y_train)
y_predict_lr = clf_lr.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_lr))
print('Precision: ', precision_score(y_test, y_predict_lr, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_lr, average='macro'))
print('F1: ', f1_score(y_test, y_predict_lr, average='macro'))

Accuracy:  0.8198976327575176
Precision:  0.7054231717337716
Recall:  0.7397833910499809
F1:  0.7193634108752104


| Модель | Accuracy | Precision | Recall | F1 |
|--------|----------|-----------|--------|----|
| Наивный Байес | 0.8218170185540628 | 0.7001475531110789 | 0.7069961308940109 | 0.7034393521925737 |
| Случайный лес | 0.8442098528470889 | 0.8172860873261675 | 0.586899995772605 | 0.6058216806076463 |
| Деревья решений | 0.7869481765834933 | 0.6487073298218808 | 0.6614764419748449 | 0.6543837922367683 |
| K-ближайших соседей | 0.8119001919385797 | 0.6377088914402347 | 0.556108273982324 | 0.5616118175018698 |
| **Метод опорных векторов** | 0.8234165067178503 | 0.7021386534193432 | 0.7065855191622272 | 0.7043069973713161 |
| **Логистическая регрессия** | 0.8198976327575176 | 0.7054231717337716 | 0.7397833910499809 | 0.7193634108752104 |

На основе приведенных метрик можно сделать следующие выводы:

1. Наивный Байес:
   - Показывает сбалансированные результаты по всем метрикам (Accuracy, Precision, Recall, F1).
   - Может быть хорошим выбором, если важно сочетание всех этих метрик.

2. Случайный лес:
   - Имеет наибольшую Accuracy (0.8442098528470889) среди представленных моделей.
   - Однако демонстрирует относительно низкий Recall (0.586899995772605), что может указывать на то, что модель пропускает некоторые положительные примеры.

3. Деревья решений:
   - Показывают самые низкие метрики качества среди представленных методов.

4. K-ближайших соседей:
   - Имеет средние результаты по сравнению с другими моделями.
   - Показывает более низкие значения Precision и Recall, что может указывать на недостаточно точные прогнозы.

5. Метод опорных векторов:
   - Демонстрирует хороший баланс между Accuracy, Precision и Recall.

6. Логистическая регрессия:
   - Показывает лучший результат по F1-мере, что может указывать на хороший баланс между Precision и Recall.

Исходя из представленных метрик, **логистическая регрессия** и **метод опорных векторов** могут быть наиболее подходящими.